<a href="https://colab.research.google.com/github/levita1905/forest-data-automation/blob/main/Dien_TKLOKH_theo_toa_do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# 📌 1. Kết nối Google Sheets và Xác thực
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads

# Xác thực Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ Kết nối thành công với Google Sheets!")

# 📌 2. Định nghĩa Google Sheets ID & Tên Sheet
SHEET_ID = "1sDEuegg7ThUirb_ZnFhfHRuUs9XAO_Xl9O7v8LWRLHo"  # 🔹 Thay thế bằng ID thực tế của bạn
SHEET_NAME_VUNG = "VungDaGiac"
SHEET_NAME_TOADO = "ToaDo"
SHEET_NAME_KETQUA = "KetQuaTraCuu"

# 📌 3. Đọc dữ liệu vùng đa giác từ Google Sheets
print("📥 Đang đọc dữ liệu vùng đa giác...")

sh_vung = gc.open_by_key(SHEET_ID).worksheet(SHEET_NAME_VUNG)
data_vung = sh_vung.get_all_values()

if len(data_vung) < 2:
    print("⚠️ Lỗi: Không có dữ liệu vùng đa giác!")
else:
    df_vung = pd.DataFrame(data_vung[1:], columns=data_vung[0])

    # Kiểm tra nếu cột WKT bị lỗi hoặc trống
    if "wkt_geom" not in df_vung.columns:
        print("⚠️ Lỗi: Cột 'wkt_geom' không tồn tại trong dữ liệu!")
    else:
        # Chuyển WKT thành Polygon
        df_vung["geometry"] = df_vung["wkt_geom"].apply(lambda wkt: loads(wkt) if wkt else None)

        # Chuyển thành GeoDataFrame
        gdf_vung = gpd.GeoDataFrame(df_vung, geometry="geometry", crs="EPSG:4326")
        print("✅ Chuyển đổi WKT thành Polygon thành công!")

# 📌 4. Đọc dữ liệu tọa độ cần kiểm tra
print("📥 Đang đọc dữ liệu tọa độ...")

sh_toado = gc.open_by_key(SHEET_ID).worksheet(SHEET_NAME_TOADO)
data_toado = sh_toado.get_all_values()

if len(data_toado) < 2:
    print("⚠️ Lỗi: Không có dữ liệu tọa độ!")
else:
    df_toado = pd.DataFrame(data_toado[1:], columns=data_toado[0])

    # Chuyển đổi X, Y sang số thực
    df_toado["X"] = pd.to_numeric(df_toado["X"], errors="coerce")
    df_toado["Y"] = pd.to_numeric(df_toado["Y"], errors="coerce")

    # Xóa các dòng có giá trị NaN trong X, Y
    df_toado = df_toado.dropna(subset=["X", "Y"])

    # Chuyển tọa độ thành GeoDataFrame
    df_toado["geometry"] = df_toado.apply(lambda row: Point(row["X"], row["Y"]), axis=1)
    gdf_toado = gpd.GeoDataFrame(df_toado, geometry="geometry", crs="EPSG:4326")

    print(f"✅ Đọc {len(df_toado)} tọa độ thành công!")

# 📌 5. Kiểm tra tọa độ thuộc vùng nào
print("📊 Đang kiểm tra tọa độ với vùng đa giác...")

if "geometry" in gdf_vung and len(gdf_vung) > 0 and len(gdf_toado) > 0:
    # Dùng sjoin và giữ lại tất cả các điểm, kể cả khi không nằm trong đa giác nào
    joined = gpd.sjoin(gdf_toado, gdf_vung, how="left", predicate="within")

    # Nếu cột từ bảng đa giác không có (vì điểm không khớp), điền giá trị mặc định là ""
    for col in ["fid", "ID_MOI", "TK", "KH", "Lo", "NGUOIKY"]:
        if col not in joined.columns:
            joined[col] = ""

    # Tuỳ chọn: Thêm thông báo nếu điểm không nằm trong vùng nào
    joined["THONG_BAO"] = joined["fid"].apply(lambda x: "" if pd.notna(x) else "Không thuộc vùng nào")

    # Chọn các cột cần thiết
    joined = joined[["X", "Y", "fid", "ID_MOI", "TK", "KH", "Lo", "NGUOIKY", "THONG_BAO"]]

    print(f"✅ Hoàn tất kiểm tra {len(joined)} tọa độ!")

    # 📌 6. Ghi kết quả vào Google Sheets
    print("📤 Đang cập nhật kết quả lên Google Sheets...")

    worksheet_ketqua = gc.open_by_key(SHEET_ID).worksheet(SHEET_NAME_KETQUA)
    values = joined.fillna("").values.tolist()
    values.insert(0, joined.columns.tolist())  # Thêm tiêu đề tự động

    worksheet_ketqua.update("A1", values)
    print("✅ Xuất kết quả thành công vào Google Sheets!")

else:
    print("⚠️ Lỗi: Không thể kiểm tra vì dữ liệu trống hoặc sai định dạng!")


✅ Kết nối thành công với Google Sheets!
📥 Đang đọc dữ liệu vùng đa giác...
✅ Chuyển đổi WKT thành Polygon thành công!
📥 Đang đọc dữ liệu tọa độ...
✅ Đọc 32 tọa độ thành công!
📊 Đang kiểm tra tọa độ với vùng đa giác...
✅ Hoàn tất kiểm tra 32 tọa độ!
📤 Đang cập nhật kết quả lên Google Sheets...


<ipython-input-3-bd6879570b36>:96: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_ketqua.update("A1", values)


✅ Xuất kết quả thành công vào Google Sheets!


0